# Feature engineering 1

This notebook contains the data processing for twitter metrics (number of tweets, number of retweets, etc.) features and sentiment analysis (number of positive, negative, etc.) features.

- Input: Tweets from candidates, Sentiment analysis
- Output: Twitter metrics and sentiment analysis related features for each candidate for each dat between 2021-01-01 and 2021-05-14

## Imports

In [7]:
import pandas as pd
import logging
import pymongo
import os
import pyathena
import dotenv
import os
import sys
from tqdm import tqdm
from collections import defaultdict
import datetime
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

#from utils.mongodb import *

In [4]:
logging.basicConfig(format='[%(asctime)s] - %(name)s - %(funcName)s - %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger(__name__)

dotenv.load_dotenv(os.path.join(module_path, '.env'))

mongo_client = pymongo.MongoClient(os.environ["MONGODB_URL"])
twitter_db = mongo_client.TwitterConstituyenteDB

conn = pyathena.connect(s3_staging_dir=os.environ["AWS_ATHENA_S3_STAGING_DIR"], 
        region_name=os.environ["AWS_REGION"])

query = """
SELECT * FROM "twitter-constituyente"."constituyentes";
"""
candidates_df = pd.read_sql(query, conn)
candidates_ids = candidates_df["user__id_str"].dropna().to_list()

[2022-01-22 23:01:19,972] - botocore.credentials - load - INFO : Found credentials in environment variables.


## Load tweets from candidates

In [5]:
fields = [
  "id_str",
  "user.id_str",
  "datetime",
  "retweet_count",
  "favorite_count",
  "retweeted_status.id_str",
  "is_quote_status",
  "in_reply_to_status_id_str",
  "entities",
  "extended_entities"
]

tweets_constituyentes = list(tqdm(twitter_db.tweets.find({
    "user.id_str": {"$in": candidates_ids },
}, fields)))

log.info(f"Tweets constituyentes: {len(tweets_constituyentes)}")

249191it [18:08, 228.95it/s] 
[2022-01-22 23:19:35,158] - __main__ - <module> - INFO : Tweets constituyentes: 249191


,_id,id_str,entities,in_reply_to_status_id_str,user,retweeted_status,is_quote_status,retweet_count,favorite_count,datetime,extended_entities
0,60ced9df9c7dd09e14630776,1383102401377415170,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,{'id_str': '1187042015382507520'},{'id_str': '1383069120464314371'},False,206,0,2021-04-16 16:58:03,NaN
1,60ced9df9c7dd09e1463077a,1382924565609844737,"{'hashtags': [], 'symbols': [], 'user_mentions...",1382851979475628033,{'id_str': '1187042015382507520'},NaN,False,0,1,2021-04-16 05:11:24,NaN
2,60ced9df9c7dd09e146307a1,1380940373992824836,"{'hashtags': [], 'symbols': [], 'user_mentions...",1380506995296460807,{'id_str': '1187042015382507520'},NaN,False,0,1,2021-04-10 17:46:56,"{'media': [{'id': 1380940370402508808, 'id_str..."
3,60ced9df9c7dd09e146307b2,1380597665650638850,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,{'id_str': '1187042015382507520'},{'id_str': '1380518028086022144'},False,3868,0,2021-04-09 19:05:08,NaN
4,60ced9df9c7dd09e14630780,1382673538738454533,"{'hashtags': [], 'symbols': [], 'user_mentions...",1382574246149419008,{'id_str': '1187042015382507520'},NaN,False,2,9,2021-04-15 12:33:55,NaN


In [8]:
tweets_constituyentes_df = pd.DataFrame(tweets_constituyentes)
tweets_constituyentes_df.head()

,_id,id_str,entities,in_reply_to_status_id_str,user,retweeted_status,is_quote_status,retweet_count,favorite_count,datetime,extended_entities
0,60ced9df9c7dd09e14630776,1383102401377415170,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,{'id_str': '1187042015382507520'},{'id_str': '1383069120464314371'},False,206,0,2021-04-16 16:58:03,NaN
1,60ced9df9c7dd09e1463077a,1382924565609844737,"{'hashtags': [], 'symbols': [], 'user_mentions...",1382851979475628033,{'id_str': '1187042015382507520'},NaN,False,0,1,2021-04-16 05:11:24,NaN
2,60ced9df9c7dd09e146307a1,1380940373992824836,"{'hashtags': [], 'symbols': [], 'user_mentions...",1380506995296460807,{'id_str': '1187042015382507520'},NaN,False,0,1,2021-04-10 17:46:56,"{'media': [{'id': 1380940370402508808, 'id_str..."
3,60ced9df9c7dd09e146307b2,1380597665650638850,"{'hashtags': [], 'symbols': [], 'user_mentions...",None,{'id_str': '1187042015382507520'},{'id_str': '1380518028086022144'},False,3868,0,2021-04-09 19:05:08,NaN
4,60ced9df9c7dd09e14630780,1382673538738454533,"{'hashtags': [], 'symbols': [], 'user_mentions...",1382574246149419008,{'id_str': '1187042015382507520'},NaN,False,2,9,2021-04-15 12:33:55,NaN


## Twitter metrics

In [9]:
def get_media_count(row):
    row["hashtags"] = len(row["entities"]["hashtags"])
    row["user_mentions"] = len(row["entities"]["user_mentions"])
    
    media_count = {}
    if pd.notna(row["extended_entities"]): 
        media_count = pd.Series(map(lambda u: u["type"], row["extended_entities"]["media"])).value_counts().to_dict()
    media_count = defaultdict(lambda: 0, media_count)
    
    row["photos"] = media_count["photo"]
    row["videos"] = media_count["video"]
    return row

tweets_constituyentes_df = tweets_constituyentes_df.apply(get_media_count, axis=1)

tweets_constituyentes_df["tweets_made"] = 1
tweets_constituyentes_df["retweets_made"] = (~tweets_constituyentes_df["retweeted_status"].isnull()).astype("int")
tweets_constituyentes_df["replies_made"] = (~tweets_constituyentes_df["in_reply_to_status_id_str"].isnull()).astype("int")
tweets_constituyentes_df["quotes_made"] = (tweets_constituyentes_df["is_quote_status"]).astype("int")

twitter_features = [
    "retweet_count",
    "favorite_count",
    "tweets_made",
    "retweets_made",
    "replies_made",
    "quotes_made",    
    "hashtags",
    "user_mentions",
    "photos",
    "videos"
]

tweets_constituyentes_df['week_monday'] = tweets_constituyentes_df['datetime'].dt.to_period('W').dt.start_time
tweets_constituyentes_df["user.id_str"] = tweets_constituyentes_df["user"].apply(lambda u: u["id_str"])
tweets_constituyentes_df["date"] = pd.to_datetime(tweets_constituyentes_df["datetime"].dt.date)

## Sentiments and emotions

In [10]:
sentiments = list(tqdm(twitter_db.sentiment_analysis.find({})))
emotions = list(tqdm(twitter_db.emotion_analysis.find({})))

sentiments_df = pd.DataFrame(sentiments)
emotions_df = pd.DataFrame(emotions)

sentiments_df["sentiment"] = sentiments_df.apply(lambda row: row["output"] if row[row["output"]] >= 0.9 else "NEU", axis=1)
emotions_df["emotion"] = emotions_df.apply(lambda row: row["output"] if row[row["output"]] >= 0.9 else "others", axis=1)

sentiments_df = sentiments_df[sentiments_df["sentiment"]!="NEU"]
sentiments_df["value"] = 1
sentiments_df = sentiments_df.pivot(index='tweet__id_str', columns='sentiment', values='value').fillna(0).reset_index()

emotions_df = emotions_df[emotions_df["emotion"]!="others"]
emotions_df["value"] = 1
emotions_df = emotions_df.pivot(index='tweet__id_str', columns='emotion', values='value').fillna(0).reset_index()



147840it [00:05, 25307.37it/s]
147840it [00:11, 12825.94it/s]


In [11]:
sentiment_emotion_features = list(sentiments_df[sentiments_df.columns[1:]]) + list(emotions_df[emotions_df.columns[1:]])
sentiment_emotion_features

['NEG', 'POS', 'anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise']

## Merging all and processing

In [12]:
tweets_constituyentes_df = (tweets_constituyentes_df
                            .merge(sentiments_df, how="left", left_on="id_str", right_on="tweet__id_str")
                            .merge(emotions_df, how="left", left_on="id_str", right_on="tweet__id_str"))

In [57]:
features_cols = ["date", "user.id_str"] + twitter_features + sentiment_emotion_features

filtered_features_df = tweets_constituyentes_df[
    (tweets_constituyentes_df["date"].dt.date >= datetime.date(2021, 1, 1)) & 
    (tweets_constituyentes_df["date"].dt.date <= datetime.date(2021, 5, 14))
][features_cols]


# getting sum of tweets and retweets made, then remove retweets
tweet_counts_df = filtered_features_df[["date", "user.id_str", "tweets_made", "retweets_made"]].groupby(["date", "user.id_str"]).sum()
log.info(f"All tweets: {len(filtered_features_df)}")
filtered_features_df = filtered_features_df[filtered_features_df["retweets_made"]==0]
log.info(f"Non-RT tweets: {len(filtered_features_df)}")

features_df = (filtered_features_df
    .drop(columns=["tweets_made", "retweets_made"])
    .fillna(0)
    .groupby(["date", "user.id_str"])
    .sum())

features_df = features_df.join(tweet_counts_df, how="outer")[twitter_features + sentiment_emotion_features]

features_df.head()


[2022-01-22 23:48:05,000] - __main__ - <module> - INFO : All tweets: 244296
[2022-01-22 23:48:05,046] - __main__ - <module> - INFO : Non-RT tweets: 145273


retweet_count  favorite_count  tweets_made  \
date       user.id_str                                                       
2021-01-01 1004762296932069377            0.0             4.0            4   
           101614836                     42.0           528.0            4   
           1016463545989566475            NaN             NaN            1   
           102121998                     11.0            24.0            9   
           104065341                      0.0             0.0            1   

                                retweets_made  replies_made  quotes_made  \
date       user.id_str                                                     
2021-01-01 1004762296932069377              1           2.0          0.0   
           101614836                        1           1.0          0.0   
           1016463545989566475              1           NaN          NaN   
           102121998                        2           7.0          0.0   
           104065341                        0           0.0          0.0   

                                hashtags  user_mentions  photos  videos  NEG  \
date       user.id_str                                                         
2021-01-01 1004762296932069377       0.0            4.0     2.0     0.0  0.0   
           101614836                 0.0            8.0     0.0     0.0  0.0   
           1016463545989566475       NaN            NaN     NaN     NaN  NaN   
           102121998                 0.0           18.0     0.0     0.0  0.0   
           104065341                 0.0            1.0     0.0     0.0  0.0   

                                POS  anger  disgust  fear  joy  sadness  \
date       user.id_str                                                    
2021-01-01 1004762296932069377  0.0    0.0      0.0   0.0  0.0      0.0   
           101614836            2.0    0.0      0.0   0.0  1.0      0.0   
           1016463545989566475  NaN    NaN      NaN   NaN  NaN      NaN   
           102121998            1.0    0.0      0.0   0.0  1.0      0.0   
           104065341            1.0    0.0      0.0   0.0  1.0      0.0   

                                surprise  
date       user.id_str                    
2021-01-01 1004762296932069377       0.0  
           101614836                 0.0  
           1016463545989566475       NaN  
           102121998                 0.0  
           104065341                 0.0

In [58]:
# reindex to have all dates in range of study

dates = pd.date_range(start='2021-01-01', end='2021-05-14')
users = list(features_df.reset_index()["user.id_str"].unique())
multi_index = pd.MultiIndex.from_product([dates, users], names=["date", "user.id_str"])
features_df_reindex = (features_df
                            .reindex(multi_index)
                            .fillna(0)
                            .astype("int")
                            .reset_index())

features_df_reindex.head()

,date,user.id_str,retweet_count,favorite_count,tweets_made,retweets_made,replies_made,quotes_made,hashtags,user_mentions,photos,videos,NEG,POS,anger,disgust,fear,joy,sadness,surprise
0,2021-01-01,1004762296932069377,0,4,4,1,2,0,0,4,2,0,0,0,0,0,0,0,0,0
1,2021-01-01,101614836,42,528,4,1,1,0,0,8,0,0,0,2,0,0,0,1,0,0
2,2021-01-01,1016463545989566475,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2021-01-01,102121998,11,24,9,2,7,0,0,18,0,0,0,1,0,0,0,1,0,0
4,2021-01-01,104065341,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0


## Save

In [65]:
features_df_reindex.to_parquet("twitter_sentiment_features.parquet", index=False)